# The Backtranslation Notebook to Rule Over Them All
Please do ignore the title, I just wanted to make it sound cool. This notebook is a simple demonstration of how to use backtranslation to improve the performance of a machine learning model. The notebook is divided into the following sections:
1. Introduction and Simple Generation
2. Applying Data Preparation/Filtering
3. Investigating Iterative Backtranslation
4. Applying a similar pipeline to a more complex model (ALMA-R)
5. Conclusion

In [9]:
# First cell in the notebook to enable autoreload of modules
%load_ext autoreload
%autoreload 2

/Users/Matey/project/nlp2/.venv/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from pathlib import Path
from transformers import FSMTForConditionalGeneration, FSMTTokenizer
from utils.data import TranslationDataModule
from utils.models import TranslationLightning
from pytorch_lightning import Trainer
from peft import LoraConfig, get_peft_model
# Isn't it so nice and clean now? I went through FOUR different ways of doing this before I thought of this one. WDWFDGFEQWDQWFGA

SRC = "de"
TGT = "en"
BATCH_SIZE = 8
MAX_LENGTH = 512

cwd = Path.cwd()
data_dir = cwd / "Data"

it_parallel = "it-parallel"
news_dataset = "train-euro-news-big"
it_mono = "it-mono"

test_folder = cwd / "tests"

mname = f"facebook/wmt19-{SRC}-{TGT}"
tokenizer = FSMTTokenizer.from_pretrained(mname)
model = FSMTForConditionalGeneration.from_pretrained(mname)

config = LoraConfig(
    r = 16,
    lora_alpha = 16,
    lora_dropout = 0.2,
    target_modules = ["v_proj", "q_proj"]
)

model = get_peft_model(model, config)
print(model.print_trainable_parameters())

# input_ids = tokenizer.encode( "Maschinelles Lernen ist großartig, oder?", return_tensors="pt")
# outputs = model.generate(input_ids)
# decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
# print(decoded)  # Machine Learning is great, isn't it?

# Copying settings from the original file though probably unneeded..
model.generation_config.length_penalty = 1.2
model.generation_config.num_beams = 5

model_pl = TranslationLightning(model, tokenizer, lr=3e-4, adam_beta=(0.9, 0.98), weight_decay=1e-4, test_folder = test_folder)

trainer = Trainer(max_steps=10,  gradient_clip_val=0.1)

Some weights of FSMTForConditionalGeneration were not initialized from the model checkpoint at facebook/wmt19-de-en and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1,179,648 || all params: 273,027,072 || trainable%: 0.4320626490841172
None


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


## Simple Generation
Here we simply load a pre-trained model and run inference on the test set. The model loaded is a pre-trained model from the [Hugging Face Transformers](https://huggingface.co/transformers/) library. 

TODO: Evaluate the model on the NEWS set and report the BLEU score.

| Model | it-parallel | NEWS |
| --- | --- | --- |
| Base | 0.0 | 0.0 |
| FT on IT-parallel | 0.0 | 0.0 |

In [3]:
data = TranslationDataModule(data_dir / it_parallel, SRC, TGT, tokenizer, batch_size=BATCH_SIZE, max_length=MAX_LENGTH)


# results = trainer.predict(model_pl, datamodule=data)

In [5]:
%%prun
# Now we can train the model
trainer.fit(model_pl, datamodule=data)
# trainer.predict(model_pl, datamodule=data)


  | Name  | Type                         | Params
-------------------------------------------------------
0 | model | FSMTForConditionalGeneration | 271 M 
-------------------------------------------------------
269 M     Trainable params
2.1 M     Non-trainable params
271 M     Total params
1,087.390 Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/Matey/project/nlp2/.venv/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
STAGE:2024-05-28 09:53:51 7995:5666521 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
/Users/Matey/project/nlp2/.venv/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

/Users/Matey/project/nlp2/.venv/lib/python3.11/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


         2443325 function calls (2367887 primitive calls) in 937.271 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     4008  417.684    0.104  417.684    0.104 {method 'index_select' of 'torch._C.TensorBase' objects}
        1  156.843  156.843  156.843  156.843 {method 'run_backward' of 'torch._C._EngineBase' objects}
     8376  107.402    0.013  107.402    0.013 {built-in method torch._C._nn.linear}
     4068   77.233    0.019   77.233    0.019 {built-in method torch.bmm}
     2004   65.097    0.032  482.782    0.241 modeling_fsmt.py:845(_reorder_buffer)
     1033   21.096    0.020   21.096    0.020 {method 'masked_fill' of 'torch._C.TensorBase' objects}
        1   17.680   17.680   17.680   17.680 {built-in method torch._C._nn.cross_entropy_loss}
     2147   17.524    0.008   17.524    0.008 {built-in method torch.cat}
     2034   16.484    0.008  187.982    0.092 modeling_fsmt.py:881(forward)
     2034    8.931    0